Adequamos as funções de avaliação de qualidade de imagem para facilitar os testes em um notebook local com o modelo BRISQUE. Na versão anterior, projetada para um endpoint, a função processava a imagem a partir de um fluxo de bytes.

A versão atual foi modificada para aceitar diretamente arquivos de imagem locais (PNG, JPG, etc.), simplificando o processo. Essa adaptação nos permite testar e validar o comportamento do modelo de forma muito mais eficiente e direta no notebook.

## Importações

In [1]:
import torch
import numpy as np
from PIL import Image, UnidentifiedImageError
from piq import brisque
import io

## Funções referentes ao modelo

In [ ]:
import torch
import numpy as np
from PIL import Image, UnidentifiedImageError

from piq import brisque

def avaliar_qualidade_de_objeto_imagem(img: Image.Image) -> dict:
    """
    Avalia a qualidade de um objeto de imagem (PIL.Image) usando o score BRISQUE.

    Args:
        img (Image.Image): O objeto de imagem da biblioteca Pillow a ser avaliado.

    Returns:
        dict: Um dicionário contendo o score e a classificação da qualidade.
    """
    try:
        img_rgb = img.convert("RGB")

        img_array = np.array(img_rgb).astype(np.float32) / 255.0
        img_tensor = torch.tensor(img_array).permute(2, 0, 1).unsqueeze(0)

        if img_tensor.shape[2] < 50 or img_tensor.shape[3] < 50:
            return {
                "erro": "A imagem é muito pequena para avaliação de qualidade.",
                "score": None,
                "qualidade": "erro",
            }

        brisque_score = brisque(img_tensor).item()

        if brisque_score < 25:
            qualidade = "boa"
        elif 25 <= brisque_score < 50:
            qualidade = "ruim"
        else:
            qualidade = "péssima"

        return {"score": round(brisque_score, 2), "qualidade": qualidade}
    
    except Exception as e:
        return {"erro": str(e), "score": None, "qualidade": "erro"}

## Caso de imagem boa

In [20]:
from PIL import Image
import requests
import io

    
with Image.open("images_brisque/vascular-lesion.jpg") as imagem_objeto:
    resultado = avaliar_qualidade_de_objeto_imagem(imagem_objeto)
    print(f"Resultado da imagem: {resultado}")


Resultado da imagem: {'score': 16.66, 'qualidade': 'boa'}


## Caso de imagem ruim

In [ ]:
from PIL import Image
import requests
import io

    
with Image.open("images_brisque/imagem_lesao_ruim.jpg") as imagem_objeto:
    resultado = avaliar_qualidade_de_objeto_imagem(imagem_objeto)
    print(f"Resultado da imagem: {resultado}")


Resultado da imagem: {'score': 31.14, 'qualidade': 'ruim'}
